In [1]:
import pandas as pd
from decimal import Decimal
import json
import numpy as np
from pprint import pprint
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.2-BETA4 (09/Jan/2022)


# Carbon Simulation - Demo-5-8 Simulator Matching Tests

Here we demonstrate the preferred setup to import and export json test files from the simulator using the new format

In [2]:
# Import Code
def load_json(filename):     
    '''
    :filename:  the convention is to specify the trade by source or trade by target in the filename i.e. tradeByTargetAmount.json
                this is necessary as the `is_by_target` field specifies the trade direction for the actionList
    '''   
    f = open(filename , "r")
    bm = f.read()
    loaded_json = json.loads(bm)
    f.close()
    return(loaded_json)

def decimalize_dict(dicta):
    deci_dict = {}
    expected_dict = {}
    for k,v in dicta.items():
        if k == 'expected':
            for i,j in dicta['expected'].items():
                expected_dict[i] = Decimal(j)
        else:    
            if k != 'token':
                v = Decimal(v)
            deci_dict[k] = v
    # deci_dict['expected'] = expected_dict
    return(deci_dict)

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['order'] = order_count
    # order0_expected = {k:v for k,v in order0['expected'].items()}
    # del order0['expected']
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    # order0_expected_df = pd.DataFrame.from_dict(order0_expected, 'index', columns=[f'{order_count}'])
    # order0_expected_df.index = ["expected_"+x for x in order0_expected_df.index]
    # order0_df = pd.concat([order0_df, order0_expected_df], axis=0)
    return(order0_df)

def parse_orders(orderList, order_count):
    orders = pd.DataFrame()
    for order in orderList:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    orders['strategy'] = orders.order
    return(orders, order_count)

# def parse_strategy(strategy, strat_count, order_count):
#     orders = pd.DataFrame()
#     orderList = strategy['orders']
#     for order in orderList:
#         order_df = parse_order(order, order_count)
#         orders = pd.concat([orders,order_df.T])
#         order_count += 1
#     orders['strategy'] = strat_count
#     strat_count += 1
#     return(orders, strat_count, order_count)

def format_json_as_df(input_json, scenario_index = 0):
    '''
    :json:             the loaded json file
    :scenario_index:   each json file has multiple scenarios, make a selection
    '''
    actionList = input_json[scenario_index]['actions']
    is_by_target = True if input_json[scenario_index]['method'] == "matchByTargetAmount" else False
    orderList = input_json[scenario_index]['orders']
    sourceToken = 'TKN0'
    targetToken = 'TKN1'
    amount = Decimal(input_json[scenario_index]['amount'])
    
    print(f'The number of scenarios is {len(input_json)}')
    print(f'You selected scenario: {scenario_index}')  
    # print(f'The number of actions in this scenario are: {len(actionList)}')

    if scenario_index >= len(input_json):
        print('Please selected a valid scenario_index')
        return()
    else:
        order_count = 0
        strats, order_count = parse_orders(orderList, order_count)
        strats['token'] = sourceToken
        return(strats, actionList, is_by_target, amount, sourceToken, targetToken)

def setup_initial_orders(strats, is_by_target, sourceToken, targetToken, matching_method):
    Sim = CarbonSimulatorUI(pair=f"{sourceToken}/{targetToken}", verbose=False, matching_method=matching_method, raiseonerror=True)
    # if is_by_target:        
    for i in strats.order:
        # print(i)
        Sim.add_strategy(
            tkn=f'{targetToken}',
            amt_sell=Decimal(str(strats.liquidity[i])),
            psell_start=strats.highestRate[i],
            psell_end=strats.lowestRate[i],
            psell_marginal=strats.marginalRate[i],
            amt_buy=None,
            pbuy_start=None,
            pbuy_end=None,
            pbuy_marginal=None,
        )
    # else:
    # #     # Sim = CarbonSimulatorUI(pair=f"{targetToken}/{sourceToken}", verbose=False, matching_method='exact', raiseonerror=True)
    #     for i in strats.order:
    #         print(i)
    #         Sim.add_strategy(
    #             tkn=f'{targetToken}',
    #             amt_sell=Decimal(str(strats.liquidity[i])),
    #             psell_start=1/strats.lowestRate[i],
    #             psell_end=1/strats.highestRate[i],
    #             psell_marginal=1/strats.marginalRate[i],
    #             amt_buy=None,
    #             pbuy_start=None,
    #             pbuy_end=None,
    #             pbuy_marginal=None,
    #         )

    initial_orders_import = Sim.state()['orders']
    return(Sim, initial_orders_import)

def process_match(Sim, is_by_target, amount, sourceToken, targetToken):
    print(f"Trading by Target: {is_by_target}; Source Token is {sourceToken}; Target Token is {targetToken}")
    if not is_by_target:
        trade_result = Sim.amm_buys(sourceToken, amount, execute=False, support_partial=True, threshold_orders = 25)['trades'] # route_trade_by_source
    else:
        trade_result = Sim.amm_sells(targetToken, amount, execute=False, support_partial=True, threshold_orders = 25)['trades'] # route_trade_by_target
    return(trade_result)

def parse_actions(actionList, is_by_target):
    actionList = [decimalize_dict(x) for x in actionList]
    actionsdf = pd.DataFrame.from_dict(actionList)
    actionsdf = actionsdf.astype(float)
    actionsdf.id = actionsdf.id.astype(int)
    actionsdf['routeix'] = [int(x)*2 for x in actionsdf.id]
    if is_by_target:
        actionsdf['p_eff'] = actionsdf.input / actionsdf.output
        actionsdf = pd.concat([actionsdf, pd.DataFrame(["A", actionsdf.input.sum(), actionsdf.output.sum(), str([x for x in actionsdf.routeix]), actionsdf.input.sum()/ actionsdf.output.sum()], index = actionsdf.columns).T], axis=0)
    else:
        actionsdf['p_eff'] = actionsdf.output / actionsdf.input
        actionsdf = pd.concat([actionsdf, pd.DataFrame(["A", actionsdf.input.sum(), actionsdf.output.sum(), str([x for x in actionsdf.routeix]), actionsdf.output.sum()/ actionsdf.input.sum()], index = actionsdf.columns).T], axis=0)
    return(actionsdf)
# def parse_actions(Sim, actionList, strats, is_by_target, sourceToken, targetToken):
#     '''
#     parse_actions processes trades from the actionList

#     :Sim:           the specific simulator environment to trade against
#     :actionList:    the list of actions provides from the test json
#     :strats:        the preprocessed dataframe of strategies from the test json
#     :is_by_target:  the specification of trading by target or by source
#     '''
#     print(f"Trading by Target: {is_by_target}; Source Token is {sourceToken}; Target Token is {targetToken}")

#     for action in actionList:
#         strategyId = int(action['strategyId']) - 1
#         targetStrategy = strats.query(f"strategy=={strategyId}")
#         tokenAmount = Decimal(action['amount'])
#         # print(strategyId, tokenAmount)
#         if not is_by_target:
#             targetOrderId = targetStrategy.query(f"token=='{targetToken}'")['order'].values[0]
#             Sim.amm_buys(sourceToken, tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
#             # print(sourceToken, targetOrderId, tokenAmount)
#         else:
#             targetOrderId = targetStrategy.query(f"token=='{targetToken}'")['order'].values[0]
#             Sim.amm_sells(targetToken, tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
#             # print(targetToken, targetOrderId, tokenAmount)

def single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, order, input_variable, final_variable, print=False):
    '''
    Asserts that the input_variable and the final_variable are exact given a defined accuracy
    '''
    if print:
        print(f"{(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" , f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")
    assert(f"{(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")

def single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, order, input_variable, final_variable, print=False):
    '''
    Asserts that the input_variable and the inverse of the final_variable are exact given a defined accuracy
    '''
    if print:
        print(f"{1/(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" , f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")
    assert(f"{1/(final_orders_import.loc[strat_index,final_variable]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][order]['expected'][input_variable]):.{accuracy}f}")

def multiLevel_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, input_variable, final_variable):
    '''
    Multi-level assertion - Asserts that the input_variable and the final_variable (or its inverse) are exact given a defined accuracy

    Specific use-case: When strategies are entered into the simulator `p_marg` may be inverted depending on how the pair is quoted.
    Such, exactly one combination should be able to be asserted.
    '''
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable, print=False)
    except:
        single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable, print=False)

    # equivalent for the other order in the strategy (strat_index increment by one and the order set to one)
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable, print=False)
    except:
        single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable, print=False)


def multiLevel_assertion_two_var(input_json, final_orders_import, scenario_index, strat_index, accuracy, input_variable, final_variable_1, final_variable_2):
    '''
    Two-variable multi-level assertion - Asserts that the input_variable and at least one of the final_variable_1 (or its inverse) or final_variable_2 (or its inverse)
    are exact given a defined accuracy

    Specific use-case: When strategies are entered into the simulator, `p_start` and `p_end` are derived from `p_a` and `p_b` however these values may be swapped or
    inverted depending on the way the pair is quoted and the order they are provided. Nevertheless, exactly one combination should be able to be asserted.
    '''
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_1, print=False)
    except:
        try:
            single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_1, print=False)
        except:
            try:
                single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_2, print=False)
            except:
                single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index, accuracy, 0, input_variable, final_variable_2, print=False)
    
    # equivalent for the other order in the strategy (strat_index increment by one and the order set to one)
    try:
        single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_1, print=False)
    except:
        try:
            single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_1, print=False)
        except:
            try:
                single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_2, print=False)
            except:
                single_assertion_flipped(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, 1, input_variable, final_variable_2, print=False)

def verify_json(filename, accuracy=7):
    input_json = load_json(filename)
    for scenario_index in range(len(input_json)):
        print("\n")
        strats, actionList, is_by_target, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
        Sim, initial_orders_import = setup_initial_orders(strats, sourceToken, targetToken)
        parse_actions(Sim, actionList, strats, is_by_target, sourceToken, targetToken)
        final_orders_import = Sim.state()['orders']
        for strat_index in final_orders_import.index[::2]:
            # print(strat_index)
            single_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, order = 0, input_variable ='liquidity', final_variable='y')
            single_assertion(input_json, final_orders_import, scenario_index, strat_index+1, accuracy, order = 1, input_variable ='liquidity', final_variable='y')
            multiLevel_assertion(input_json, final_orders_import, scenario_index, strat_index, accuracy, 'marginalRate', 'p_marg')
            multiLevel_assertion_two_var(input_json, final_orders_import, scenario_index, strat_index, accuracy, 'lowestRate', 'p_start', 'p_end')
            multiLevel_assertion_two_var(input_json, final_orders_import, scenario_index, strat_index, accuracy, 'highestRate', 'p_start', 'p_end')

    print("\n----------All Tests Passed!----------\n")

def verify_json2(filename, accuracy=12):
    misses = {}
    misses['liquidity'] = {}
    misses['marginalPrice'] = {}
    input_json = load_json(filename)
    for scenario_index in range(len(input_json)):
        print("\n")
        strats, actionList, is_by_target, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
        Sim, initial_orders_import = setup_initial_orders(strats, sourceToken, targetToken)
        parse_actions(Sim, actionList, strats, is_by_target, sourceToken, targetToken)
        final_orders_import = Sim.state()['orders']
        sub_misses = []
        sub_misses_p = []
        for strat_index in final_orders_import.index[::2]:
            if not f"{final_orders_import.y[strat_index]:.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['liquidity']):.{accuracy}f}":
                sub_misses += [(f"{final_orders_import.y[strat_index]:.{accuracy}f}", f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['liquidity']):.{accuracy}f}")]
            
            if not f"{final_orders_import.y[strat_index+1]:.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['liquidity']):.{accuracy}f}":
                sub_misses += [(f"{final_orders_import.y[strat_index+1]:.{accuracy}f}", f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['liquidity']):.{accuracy}f}")]

            if f"{(final_orders_import.p_marg[strat_index]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['marginalRate']):.{accuracy}f}":
                pass
            elif f"{1/(final_orders_import.p_marg[strat_index]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['marginalRate']):.{accuracy}f}":
                 pass
            else:
                sub_misses_p += [(f"{1/(final_orders_import.p_marg[strat_index]):.{accuracy}f}", f"{(final_orders_import.p_marg[strat_index]):.{accuracy}f}",  f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][0]['expected']['marginalRate']):.{accuracy}f}")]

            if f"{(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['marginalRate']):.{accuracy}f}":
                pass
            elif f"{1/(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}" == f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['marginalRate']):.{accuracy}f}":
                pass
            else:
                sub_misses_p += [(f"{1/(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}", f"{(final_orders_import.p_marg[strat_index+1]):.{accuracy}f}", f"{float(input_json[scenario_index]['strategies'][strat_index//2]['orders'][1]['expected']['marginalRate']):.{accuracy}f}")]
        misses['liquidity'][scenario_index] = sub_misses
        misses['marginalPrice'][scenario_index] = sub_misses_p
    return(misses)

# Export Code
def generate_json_actions_new(Sim, trade_a):

    json_actions = {}
    sub_actions_list = []

    if trade_a['success'] & trade_a['trades'].exec.all():
        subdf = trade_a['trades'].query('subid!="A"').copy()
        subdf.reset_index(inplace=True, drop=True)
        trade_is_by_target = trade_a['is_by_target']
        tkn = trade_a['tkn_sold']
        sourceToken = Sim.carbon_pair.other(tkn)
        targetToken = tkn

        print(f'trade_is_by_target: {trade_is_by_target}')
        for i in subdf.index:
            sub_actions = {}
            sub_actions['strategyId'] = f"{int((subdf.routeix[i]//2) + 1)}" # plus one to encode for contracts
            if trade_is_by_target:
                sub_actions['amount'] = f"{subdf.amt1[i]:.12f}"
            else:
                sub_actions['amount'] = f"{subdf.amt2[i]:.12f}"
            sub_actions_list += [sub_actions]
            
    json_actions['tradeActions'] = sub_actions_list
    return(json_actions, trade_is_by_target, sourceToken, targetToken)

def generate_json_strategies_new(initial_orders, final_orders):
    json_strats = {}
    json_orders = {}
    json_orders_list = []
    sub_dict_list = []
    for i in initial_orders.index[::2]:
        json_orders = {} 
        sub_dict0 = {}
        sub_dict0['token'] = initial_orders.tkn[i]
        sub_dict0['liquidity'] = f"{initial_orders.y[i]:.12f}"
        sub_dict0['lowestRate'] = f"{1/(initial_orders.p_end[i]):.12f}"  # - 0.000000001
        sub_dict0['highestRate'] = f"{(1/initial_orders.p_start[i]):.12f}" # + 0.000000001
        sub_dict0['marginalRate'] = f"{(1/initial_orders.p_marg[i]):.12f}"

        sub_dict0['expected'] = {}
        sub_dict0['expected']['liquidity'] = f"{final_orders.y[i]:.12f}"
        sub_dict0['expected']['lowestRate'] = f"{1/(final_orders.p_end[i]):.12f}"  # - 0.000000001
        sub_dict0['expected']['highestRate'] = f"{(1/final_orders.p_start[i]):.12f}" # + 0.000000001
        sub_dict0['expected']['marginalRate'] = f"{(1/final_orders.p_marg[i]):.12f}"
        
        sub_dict1 = {}   
        
        sub_dict1['token'] = initial_orders.tkn[i+1]
        sub_dict1['liquidity'] = f"{initial_orders.y[i+1]:.12f}" 
        sub_dict1['lowestRate'] = f"{((initial_orders.p_end[i+1])):.12f}" # + 0.000000001
        sub_dict1['highestRate'] = f"{((initial_orders.p_start[i+1])):.12f}" # - 0.000000001
        sub_dict1['marginalRate'] = f"{(initial_orders.p_marg[i+1]):.12f}"
        
        sub_dict1['expected'] = {}
        sub_dict1['expected']['liquidity'] = f"{final_orders.y[i+1]:.12f}" 
        sub_dict1['expected']['lowestRate'] = f"{((final_orders.p_end[i+1])):.12f}" # + 0.000000001
        sub_dict1['expected']['highestRate'] = f"{((final_orders.p_start[i+1])):.12f}" # - 0.000000001
        sub_dict1['expected']['marginalRate'] = f"{(final_orders.p_marg[i+1]):.12f}"

        json_orders['orders'] = [sub_dict0,sub_dict1]
        json_orders_list += [json_orders]
    json_strats['strategies'] = json_orders_list
    return(json_strats)

def format_simulator_json_new(json_strats, json_actions, trade_is_by_target, sourceToken, targetToken):
    json_simulator = []
    simulator_setup = {}
    simulator_setup['tradeByTargetAmount'] = True if trade_is_by_target else False
    simulator_setup['sourceToken'] = sourceToken
    simulator_setup['targetToken'] = targetToken
    simulator_setup.update({**json_strats, **json_actions})
    json_simulator += [simulator_setup]
    return(json_simulator)

def get_compare_actionsdf(actionsdf):
    compare_actionsdf = actionsdf.query("id=='A'")
    compare_actionsdf = compare_actionsdf[['input','output','routeix','p_eff']].copy()
    return(compare_actionsdf)

def get_sub_result(trade_result, is_by_target):
    if not is_by_target:
        sub_result = trade_result.query("subid=='A'")[['amt2', 'amt1', 'routeix', 'price']]
    else:
        sub_result = trade_result.query("subid=='A'")[['amt1', 'amt2', 'routeix', 'price']]
    sub_result.columns = ['input','output','routeix','p_eff']
    return(sub_result)

def evaluate_delta(intial_amount, final_amount):
    delta = final_amount - intial_amount
    delta_perc = delta / intial_amount
    return(delta, delta_perc)

def compare_results(sub_result, compare_actionsdf):
    comparison_results = {}
    simple_results = {}
    for col in compare_actionsdf.columns:
        if col != 'routeix':
            var1 = sub_result.loc[0, col]
            var2 = compare_actionsdf.loc[0, col]
            delta, delta_perc = evaluate_delta(var1, var2)
            comparison_results[col] = [var1, var2, delta_perc]
            simple_results[col] = abs(delta_perc)
            # assert(abs(delta_perc) < 1E-05)
        else:
            set1 = set(eval(sub_result.routeix[0]))
            set2 = set(eval(compare_actionsdf.routeix[0]))
            set_bool = set1 != set2 #since False evals to 0 it makes threshold testing easier
            comparison_results['routeix'] = [set1-set2, set2-set1, set_bool]
            simple_results['routeix'] = set_bool
            # assert(set(eval(sub_result.routeix[0])) == set(eval(compare_actionsdf.routeix[0])))
    return(comparison_results, simple_results)

def verify_match_json(filename, matching_method = 'exact'):
    input_json = load_json(filename)
    full_comparison = {}
    simple_comparison = {}
    for scenario_index in range(len(input_json)):
        strats, actionList, is_by_target, amount, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
        Sim, initial_orders_import = setup_initial_orders(strats, is_by_target, sourceToken, targetToken, matching_method)
        trade_result = process_match(Sim, is_by_target, amount, sourceToken, targetToken)
        actionsdf = parse_actions(actionList, is_by_target)
        sub_result = get_sub_result(trade_result, is_by_target)
        compare_actionsdf = get_compare_actionsdf(actionsdf)
        comparison_results, simple_results = compare_results(sub_result, compare_actionsdf)
        full_comparison[scenario_index] = comparison_results
        simple_comparison[scenario_index] = simple_results
    return(full_comparison, simple_comparison)

def full_comparison_threshold(full_comparison, accuracy):
    for i in range(len(full_comparison)):
        for k in full_comparison[i].keys():
            if full_comparison[i][k][2] > accuracy:
                print(i, k, full_comparison[i][k])

# To Import Json and Verify against simulator
- Asserts that expected liquidity and marginalRate match the expected results to a given accuracy threshold

In [3]:
# filename = '../benchmark/ArbitraryMatch.json'
# matching_method = 'alpha'

# input_json = load_json(filename)
# full_comparison = {}
# simple_comparison = {}
# for scenario_index in range(len(input_json))[127:128]:
#     strats, actionList, is_by_target, amount, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
#     Sim, initial_orders_import = setup_initial_orders(strats, is_by_target, sourceToken, targetToken, matching_method)
#     # trade_result = process_match(Sim, is_by_target, amount, sourceToken, targetToken)
#     # actionsdf = parse_actions(actionList, is_by_target)
#     # sub_result = get_sub_result(trade_result, is_by_target)
# print(Sim.state()['orders'].y.sum())
# Sim.state()['orders']


In [4]:
# Sim.amm_sells(targetToken, amount, execute=False, support_partial=True, threshold_orders = 10)['trades']

In [5]:
# filename = '../benchmark/ArbitraryMatch.json'
# matching_method = 'alpha'

# input_json = load_json(filename)
# full_comparison = {}
# simple_comparison = {}
# for scenario_index in range(len(input_json))[128:129]:
#     strats, actionList, is_by_target, amount, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
#     Sim, initial_orders_import = setup_initial_orders(strats, is_by_target, sourceToken, targetToken, matching_method)
#     # trade_result = process_match(Sim, is_by_target, amount, sourceToken, targetToken)
#     # actionsdf = parse_actions(actionList, is_by_target)
#     # sub_result = get_sub_result(trade_result, is_by_target)
# print(Sim.state()['orders'].y.sum())
# Sim.state()['orders']


In [6]:
# Sim.amm_sells(targetToken, amount, execute=False, support_partial=True, threshold_orders = 10)['trades']

In [7]:
# filename = '../benchmark/ArbitraryMatch.json'
# matching_method = 'exact'

# input_json = load_json(filename)
# full_comparison = {}
# simple_comparison = {}
# for scenario_index in range(len(input_json))[128:129]:
#     strats, actionList, is_by_target, amount, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
#     Sim, initial_orders_import = setup_initial_orders(strats, is_by_target, sourceToken, targetToken, matching_method)
#     # trade_result = process_match(Sim, is_by_target, amount, sourceToken, targetToken)
#     # actionsdf = parse_actions(actionList, is_by_target)
#     # sub_result = get_sub_result(trade_result, is_by_target)
# print(Sim.state()['orders'].y.sum())
# Sim.state()['orders']


In [8]:
# Sim.amm_sells(targetToken, amount, execute=False, support_partial=True, threshold_orders = 10)['trades']

In [9]:
# Sim.amm_sells(targetToken, 818178.6984295943, execute=True, use_positions=[0])['trades']

In [10]:
# Sim.state()['orders']

In [11]:
# Sim.state()['orders'].p_marg.values[0]

In [12]:
# # initial scenario
# y_int = 1481481
# pa = 1
# pb = 0.5
# pm = 0.9968353059410049

# y = y_int * (np.sqrt(pm) - np.sqrt(pb)) / (np.sqrt(pa) - np.sqrt(pb))
# y

In [13]:
# # want to move price to P = pm = 0.7
# y_int = 1481481
# pa = 1
# pb = 0.5
# pm = 0.7 # P

# y = y_int * (np.sqrt(pm) - np.sqrt(pb)) / (np.sqrt(pa) - np.sqrt(pb))
# y

In [14]:
# delta y = amount of liquidity to remove i.e. depth associated with this order
# 1473471.000000237 - 655292.3015706427

In [15]:
# filename = '../benchmark/ArbitraryMatch.json'
# full_comparison, simple_comparison = verify_match_json(filename, 'exact')

In [16]:
# full_comparison_threshold(full_comparison, accuracy = 1e-05)

In [17]:
# simple_comparison_df = pd.DataFrame.from_dict(simple_comparison).T
# simple_comparison_df.sort_values(by='p_eff', ascending=False)[:20]

In [18]:
# break

In [19]:
filename = '../benchmark/ArbitraryMatch.json'
full_comparisona, simple_comparisona = verify_match_json(filename, 'alpha')

The number of scenarios is 200
You selected scenario: 0
Trading by Target: False; Source Token is TKN0; Target Token is TKN1
The number of scenarios is 200
You selected scenario: 1
Trading by Target: False; Source Token is TKN0; Target Token is TKN1
The number of scenarios is 200
You selected scenario: 2
Trading by Target: False; Source Token is TKN0; Target Token is TKN1
The number of scenarios is 200
You selected scenario: 3
Trading by Target: False; Source Token is TKN0; Target Token is TKN1
The number of scenarios is 200
You selected scenario: 4
Trading by Target: False; Source Token is TKN0; Target Token is TKN1
The number of scenarios is 200
You selected scenario: 5
Trading by Target: False; Source Token is TKN0; Target Token is TKN1
The number of scenarios is 200
You selected scenario: 6
Trading by Target: False; Source Token is TKN0; Target Token is TKN1
The number of scenarios is 200
You selected scenario: 7
Trading by Target: False; Source Token is TKN0; Target Token is TKN1


In [20]:
full_comparison_threshold(full_comparisona, accuracy = 1e-05)

4 input [13542640.904968658, 13547034.0, 0.00032438983372362444]
4 output [73999707.88079342, 74004099.0, 5.93396829843061e-05]
5 input [13525384.349667396, 13532034.0, 0.0004916422454765306]
5 output [73927517.14957415, 73934169.0, 8.99780038925958e-05]
6 input [13501661.473452952, 13511036.0, 0.0006943239219469761]
6 output [73812923.18325743, 73822305.0, 0.00012710263105657945]
7 input [13470475.018933961, 13483038.0, 0.0009326308870608078]
7 output [73641966.86693414, 73654546.0, 0.00017081473514407965]
8 input [13430828.750617139, 13447038.0, 0.0012068688897634967]
8 output [73398703.32885556, 73414944.0, 0.0002212664585050783]
9 input [13381729.839121018, 13402038.0, 0.0015176035627032464]
9 output [73065206.47980364, 73085568.0, 0.00027867600979122084]
11 input [1101922.9697949186, 1101986.0, 5.7200191673271426e-05]
11 output [14407965.73298163, 14408446.0, 3.333343702164277e-05]
12 input [1098758.223728805, 1098986.0, 0.00020730335962530888]
12 output [14377683.95581059, 143794

In [21]:
simple_comparison_dfa = pd.DataFrame.from_dict(simple_comparisona).T
simple_comparison_dfa.sort_values(by='p_eff', ascending=False)[:20]

,input,output,routeix,p_eff
99,0.00614,0.003746,False,0.002379
59,0.006117,0.003734,False,0.002369
19,0.006073,0.003709,False,0.00235
39,0.005988,0.003659,False,0.002316
79,0.005823,0.003563,False,0.002246
98,0.004601,0.002796,False,0.001796
58,0.004584,0.002787,False,0.001789
18,0.004552,0.002768,False,0.001775
38,0.004488,0.002731,False,0.001749
78,0.004364,0.00266,False,0.001696


In [22]:
# pd.set_option('display.float_format', lambda x: '%.5f' % x)
break


SyntaxError: 'break' outside loop (524577271.py, line 2)

- Tests every expected liquidity and marginalRate match the expected results against a given accuracy threshold and returns a dict of the failures

In [ ]:
%%capture
misses = verify_json2(filename, accuracy=12)

In [ ]:
print('Liquidity')
for i in misses['liquidity'].keys():
    if len(misses['liquidity'][i])>0:
        print(i, misses['liquidity'][i])
print('\n')
print('Marginal Prices')
for i in misses['marginalPrice'].keys():
    if len(misses['marginalPrice'][i])>0:
        print(i, misses['marginalPrice'][i])

# To Create New Scenario and Export to Json

## Initialize a simulator and add some strategies

In [ ]:
# New Sim
Sim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='exact', raiseonerror=True)

In [ ]:
# Add strategies
Sim.add_strategy("ETH", 10, 2000, 2500, 10000, 1040, 1020, "ETH/USDC", 2100, 1030)
Sim.add_strategy("ETH", 10, 2010, 2810, 10000, 1510, 1210, "ETH/USDC", 2100, 1250)
Sim.add_strategy("ETH", 10, 2020, 2220, 10000, 1060, 1050, "ETH/USDC", 2200, 1055)
Sim.add_strategy("ETH", 10, 2030, 2060, 10000, 1750, 1055, "ETH/USDC", 2035, 1700)
Sim.add_strategy("ETH", 10, 2040, 2080, 10000, 1250, 1040, "ETH/USDC", 2050, 1050)

# Snapshot the simulator orders
initial_orders = Sim.state()['orders']

In [ ]:
initial_orders

## Do a trade and process it into an actionList

In [ ]:
# Do a trade
trade_a = Sim.amm_sells('ETH', 29, execute=True) # route_trade_by_target
# trade_a = Sim.amm_buys('ETH', 15, execute=True) # route_trade_by_source
# trade_a = Sim.amm_sells('USDC', 50000, execute=True) # route_trade_by_target
# trade_a = Sim.amm_buys('USDC', 30000, execute=True) # route_trade_by_source

# Generate the actions component of the json
json_actions, trade_is_by_target, sourceToken, targetToken  = generate_json_actions_new(Sim, trade_a)
sourceToken, targetToken

In [ ]:
# Snapshot the orders
final_orders = Sim.state()['orders']
final_orders

## Generate the strategies component of the json

In [ ]:
# Generate the strategies component of the json
json_strats = generate_json_strategies_new(initial_orders, final_orders)

## Combine and format the jsons

In [ ]:
json_simulator = format_simulator_json_new(json_strats, json_actions, trade_is_by_target, sourceToken, targetToken)
# pprint(json_simulator, sort_dicts=False)

## Optionally Save

In [ ]:
from datetime import datetime
str_date = datetime.today().strftime("%Y%m%d")
unique_id = datetime.now().strftime("%f")
    
export_filename = f"{str_date}_test_{unique_id}.json"

out_file = open(export_filename, "w")
json.dump(json_simulator, out_file, indent=4)
out_file.close()

print(f'Saved as {export_filename}')

# To Import json and Evaluate Step-wise

First we load the json test file and return a df of the strategies and a list of actions

** If you want to use a specific scenario you need to specify the scenario_index here **

In [ ]:
# Specify the scenario_index for a different scenario
filename=export_filename
# filename = 'ArbitraryTrade.json'
scenario_index = 0

input_json = load_json(filename)
strats, actionList, is_by_target, sourceToken, targetToken = format_json_as_df(input_json, scenario_index = scenario_index)
strats

# Initialize the Simulator and create the orders
Here the orders are entered into the simulator and the prices for one side of the strategy are flipped to coincide with the price convention

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

Sim, initial_orders_import = setup_initial_orders(strats, sourceToken, targetToken)
initial_orders_import

# Parse the actions to perform the trades

In [ ]:
parse_actions(Sim, actionList, strats, is_by_target, sourceToken, targetToken)

Sim.state()['trades']

# Display the status of the orders post-trade

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders_import = Sim.state()['orders']
final_orders_import